<a href="https://colab.research.google.com/github/SnoopiACK/DientesMask/blob/master/demo_deteccion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install detectron2

In [0]:
# install dependencies: (use cu100 because colab is on CUDA 10.0)
!pip install -U torch==1.4+cu100 torchvision==0.5+cu100 -f https://download.pytorch.org/whl/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__
!gcc --version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu100/index.html

In [0]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup:
# Setup detectron2 logger
import torch, torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# Train on a custom dataset

In [3]:
import os
repo_url = 'https://github.com/SnoopiACK/DientesMask'
!git clone {repo_url}
#%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
%cd {repo_dir_path}
!git pull

Cloning into 'DientesMask'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (144/144), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 701 (delta 44), reused 79 (delta 14), pack-reused 557
Receiving objects: 100% (701/701), 299.22 MiB | 35.59 MiB/s, done.
Resolving deltas: 100% (232/232), done.
Checking out files: 100% (444/444), done.
/content/DientesMask
Already up to date.


In [0]:
import glob
import json
from detectron2.structures import BoxMode


#dado el numero de diente, saber si es incisivo, canino, premolar, molar
def get_tooth_type(tooth_number):
  if len(tooth_number) == 2:
    c = tooth_number[1]
  else:
    c = tooth_number 
  if (c == '1') or (c == '2'):
    tooth_type = 'Incisivo'
  elif (c == '3'):
    tooth_type = 'Canino'
  elif (c == '4') or (c == '5'):
    tooth_type = 'Premolar'
  else:
    tooth_type = 'Molar'  
  return tooth_type
  
#encoding de tipo de diente
def tooth_type_encoder(tooth_type):
  encoder_dict = {
      "Incisivo":0,
      "Canino":1,
      "Premolar":2,
      "Molar":3
  }
  return encoder_dict[tooth_type]

#function that makes the data dictionaries for detectron to use
#jsons_path: folder full of .json files
#images_path: folder where the dataset images are
def make_data_dicts(jsons_path, images_path):
    dataset_dicts = []
    files = glob.glob(jsons_path + "/*")
    jsons = [file for file in files if '.json' in file]
    #images = [file for file in files if not '.json' in file]
    for json_filename in jsons:
      json_file = json.load( open(json_filename, encoding = 'cp1252' ) )
      record = {}
      record["file_name"] = images_path + '/' + json_file['imagePath']
      record["image_id"] = json_file['imagePath']
      record["height"] = json_file['imageHeight']
      record["width"] = json_file['imageWidth']
      objs = []
      for shape in json_file['shapes']:
        shape_category = tooth_type_encoder( get_tooth_type(shape['label']) )
        obj = {
          "bbox": [round(shape['points'][0][0]), round(shape['points'][0][1]), round(shape['points'][1][0]), round(shape['points'][1][1])],
          "bbox_mode": BoxMode.XYXY_ABS,  
          "category_id": shape_category,
        }
        objs.append(obj)
      record["annotations"] = objs
      dataset_dicts.append(record)
    return dataset_dicts


from detectron2.data import DatasetCatalog, MetadataCatalog
for d in ["train", "test"]:
    DatasetCatalog.register("data_" + d, lambda d=d: make_data_dicts("data/jsons_comparacion_de_redes/jsons_dientes_enumerados/" + d, "data/images"))
    MetadataCatalog.get("data_" + d).set(thing_classes=["I","C","P","M"])
dataset_metadata = MetadataCatalog.get("data_train")

To verify the data loading is correct, let's visualize the annotations of randomly selected samples in the training set:



In [0]:
dataset_dicts = make_data_dicts("data/jsons_comparacion_de_redes/jsons_dientes_enumerados/train", "data/images")
for d in random.sample(dataset_dicts, 2):
    print(d)
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=dataset_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

In [0]:
from detectron2.evaluation import COCOEvaluator
from detectron2.engine import DefaultTrainer
class CustomTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("custom_eval", exist_ok=True)
        output_folder = "custom_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

## Train!

In [11]:
%rm -rf output
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ('data_train',)
cfg.DATASETS.TEST = ('data_test',)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 1000  #  iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # number of classes
cfg.TEST.EVAL_PERIOD = 10


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CustomTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[05/23 16:21:26 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [0]:
# Look at training curves in tensorboard:
%load_ext tensorboard
%tensorboard --logdir output

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [0]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("data_test", )
predictor = DefaultPredictor(cfg)

Then, we randomly select several samples to visualize the prediction results.

In [0]:

!dir
from detectron2.utils.visualizer import ColorMode
dataset_dicts = make_data_dicts("data/jsons_comparacion_de_redes/jsons_dientes_enumerados/test", "data/images")
for d in random.sample(dataset_dicts, 6):   
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata = dataset_metadata, 
                   scale = 0.5, 
                   #instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(v.get_image()[:, :, ::-1])

We can also evaluate its performance using AP metric implemented in COCO API.
This gives an AP of ~70%. Not bad!

In [17]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("data_test", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "data_test")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test

[05/23 17:05:58 d2.data.common]: Serializing 6 elements to byte tensors and concatenating them all ...
[05/23 17:05:58 d2.data.common]: Serialized dataset takes 0.01 MiB
[05/23 17:05:58 d2.evaluation.evaluator]: Start inference on 6 images
[05/23 17:06:01 d2.evaluation.evaluator]: Total inference time: 0:00:00.354398 (0.354398 s / img per device, on 1 devices)
[05/23 17:06:01 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.297983 s / img per device, on 1 devices)
[05/23 17:06:01 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[05/23 17:06:01 d2.evaluation.coco_evaluation]: Saving results to ./output/coco_instances_results.json
[05/23 17:06:01 d2.evaluation.coco_evaluation]: Evaluating predictions ...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.09s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=

OrderedDict([('bbox',
              {'AP': 68.85525085140134,
               'AP-C': 68.23238990565723,
               'AP-I': 68.13691075013793,
               'AP-M': 68.84698164263861,
               'AP-P': 70.20472110717161,
               'AP50': 96.67084993706713,
               'AP75': 86.90205875285112,
               'APl': 68.89478178553917,
               'APm': 69.65234559170203,
               'APs': 85.04950495049505})])

## Copy all results in drive

In [23]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)
#%mkdir "/gdrive/My Drive/models/faster_rcnn" #o el directorio que se quiera
%cp -r "output/." "/gdrive/My Drive/models/faster_rcnn"

Mounted at /gdrive


#Predicting an image and making a .json with it

In [0]:
import base64

#function to make a .json out of a single image
def predict_img_and_make_json(img_path, predictor):
  #img file name without dirname
  img_name = os.path.basename(img_path)
  #make base64 of image (its a field in the .json)
  encoded = base64.b64encode(open(img_path, "rb").read())
  #predict
  img = cv2.imread(img_path)
  outputs = predictor(img)
  #make the .json out of the prediction
  inst = outputs['instances']
  height, width = inst.image_size
  inst_fields = inst.get_fields()
  #boxes found
  pred_boxes = inst_fields['pred_boxes']
  #classes found (same length as pred_boxes)
  pred_classes = inst_fields['pred_classes']
  #list containing all the boxes that were found
  shapes_list = []
  for i in range(len(pred_boxes)):
    box = pred_boxes[i].tensor[0].cpu().numpy()
    xmin, ymin, xmax, ymax = int(box[0]), int(box[1]), int(box[2]), int(box[3])
    #the label that will be stored in the .json is the numeric label corresponding
    #to the "category_id" assigned in the dataset. It could be possible to use
    #the metadata of the dataset to get the class name corresponding to the category_id
    label = int( pred_classes[i].cpu().numpy() )
    #dictionary for the box being processed
    shape_dict = {"line_color":None, "fill_color":None, "label":str(label),
                  "points":[[xmin,ymin],[xmax,ymax]], "group_id":None, "shape_type":"rectangle","flags": {}}
    shapes_list.append(shape_dict)
  
  #make the .json
  output_dict = {
    "version": "4.2.9",
    "flags": {},
    "shapes": shapes_list,
    "imagePath":img_name,
    "imageData": str(encoded)[1::],
    "imageHeight": height,
    "imageWidth": width,
    "lineColor": [
      0,
      255,
      0,
      128
    ],
    "fillColor": [
      255,
      0,
      0,
      128
    ]
  }

  #write the .json
  output_json_file_name = img_path.split('.')[0] + '.json'
  with open(output_json_file_name, 'w') as outfile:
    json.dump(output_dict, outfile, indent=4)

#function to predict and make .jsons of all the images in a folder
def predict_folder_and_make_jsons(folder_path, predictor):
  imgs_list = os.listdir(folder_path)
  for img_file in imgs_list:
    img_path = os.path.join(folder_path, img_file)
    try:
      predict_img_and_make_json(img_path, predictor)
    except:
      continue

predict_folder_and_make_jsons('data/images_05_04', predictor)


In [0]:
#!zip -r /content/file.zip /content/X-ray-object-detection/data/images_05_04